In [ ]:
base_dir = "/sbgenomics/workspace/"
curie_map = "BDCHM_Curie_Remap.tsv"

# input_dir = "/sbgenomics/output-files/TSV_output/COPDGene-v6-c1/"
# output_dir = "/sbgenomics/output-files/TSV_output/COPDGene-v6-c1-remapped/"
# ext = "tsv"

# input_dir = "/sbgenomics/output-files/TSV_output/CARDIA-v3-c1/"
# output_dir = "/sbgenomics/output-files/TSV_output/CARDIA-v3-c1-remapped/"
# ext = "tsv"

# input_dir = "/sbgenomics/output-files/TSV_output/CHS-v7-c1/"
# output_dir = "/sbgenomics/output-files/TSV_output/CHS-v7-c1-remapped/"
# ext = "tsv"

# input_dir = "/sbgenomics/output-files/TSV_output/FHS-v31-c1/"
# output_dir = "/sbgenomics/output-files/TSV_output/FHS-v31-c1-remapped/"
# ext = "tsv"

input_dir = "/sbgenomics/output-files/TSV_output/HCHS-SOL-v1-c1"
output_dir = "/sbgenomics/output-files/TSV_output/HCHS-SOL-v1-c1-remapped/"
ext = "tsv"

In [ ]:
%%bash -s "$base_dir" "BDCHM Variable Mapping - CURIE Lookup.tsv" "$curie_map"
# --- Extract mapping columns from the CURIE lookup spreadsheet ---
tr -d '\r' < "$1/$2" > "$1/$2.tmp"
mv "$1/$2.tmp" "$1/$2"
cut -f2,5 "$1/$2" > "$1/$3"

In [ ]:
%%bash -s "$base_dir/$curie_map" "$input_dir" "$output_dir" "$ext"
# --- Remap all TSV file CURIEs the input directory to output directory---
mkdir -p "$3"
for f in "$2"/*."$4"; do
  # $1..$4 are bash args; inside awk $1/$2 are awk fields (not bash $1/$2)
  awk 'FNR==NR{map[$1]=$2; next} {for(k in map) gsub(k,map[k]); print}' "$1" "$f" > "$3/$(basename "$f")"
done

In [ ]:
# --- List unique CURIEs in output files ---
!grep -rh -oE '\b[A-Z][A-Za-z0-9]*:[A-Za-z0-9]+\b' "$output_dir" | sort -u

In [ ]:
%%bash -s "$output_dir"
# --- Validation: confirm all CURIEs have been remapped ---
curies=$(grep -rh -oE '\b[A-Z][A-Za-z0-9]*:[A-Za-z0-9]+\b' "$1" | sort -u)

if [ -z "$curies" ]; then
    echo "All CURIEs successfully remapped."
else
    echo "Some CURIEs remain in the output files:"
    grep -rE -l '\b[A-Z][A-Za-z0-9]*:[A-Za-z0-9]+\b' "$1"
    echo "$curies"
fi
